### Packages 

这里是之后用到的库 

#### Pandas
#### Numpy
#### Matplotlib 
#### Math
#### Sklearn
#### Warning

Statquest: 兄弟们有时间大家可以看看statquest这个channel 对几乎所有的统计和machine learning的概念解释的特别通俗易懂 非常有帮助
对以后找工作和面试帮助应该非常大 

https://www.youtube.com/@statquest/playlists 

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn 
import numpy as np 
from math import sin, cos, sqrt, atan2, radians

import sklearn
import warnings

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import classification_report
from sklearn import metrics

%matplotlib inline

In [2]:
df_tourney = pd.read_csv("NCAA_Tourney_2002_2022.csv").query("season!=2019").reset_index(drop=True)

### Mix Labels 

#### For the dataset provided, team1 have won all of the games, which is a vital flaw if we put the dataset into the model. Therefore, it's necessary for us to mix the labels, which contains both 0 and 1. 

#### Now, we try to random select rows from the dataset, and switch the position of team_1 and team_2. 

In [3]:
np.random.seed(1313)

remove_n = 650
random_indices = np.random.choice(df_tourney.index, remove_n, replace=False)

df1 = df_tourney.drop(random_indices)
df2 = df_tourney.iloc[random_indices, :] 

#### team_1 -> team_2, team_1 -> team_1

In [4]:
df2.columns = df2.columns.str.replace("team1","team3",regex=True)

In [5]:
df2.columns = df2.columns.str.replace("team2","team1",regex=True)

In [6]:
df2.columns = df2.columns.str.replace("team3","team2",regex=True)

In [7]:
print(df2.head(5))

      team2_id  team2_score  team1_id  team1_score WLoc  num_ot  \
1208      1393           57      1395           52    N       0   
34        1181           78      1168           61    N       0   
1113      1291           67      1309           66    N       0   
786       1243           70      1379           64    N       0   
951       1276           79      1400           65    N       0   

     team2_position team1_position  team2_seed  team1_seed  ...  team2_adjoe  \
1208           X11b            X06          11           6  ...     103.0000   
34              X02            X15           2          15  ...     119.3570   
1113           W16a           W16b          16          16  ...      98.9643   
786             W08            W09           8           9  ...     108.6262   
951             Y02            Y07           2           7  ...     122.2637   

      team2_de team2_adjde team1_tempo team1_adjtempo  team1_oe team1_adjoe  \
1208   94.1000     98.3000     70.300

In [8]:
df  = pd.concat([df2, df1]).reset_index(drop=True)

### Data Munipulation

#### Create New Features (建立新的特征)

之后我们可以在这里面创建一些新的variable 

Meeting的时候我们可以找一些其他的数据构建新的variable 

In [9]:
# Seed Difference 
df["sead_diff"] = df["team1_seed"] - df["team2_seed"]

In [10]:
# Regions of each team 
df['team1_region'] = df["team1_position"].str[:1] 
df['team2_region'] = df["team2_position"].str[:1]

In [11]:
df.drop(columns=['team1_position', 'team2_position'], inplace = True)

In [12]:
# Expected Win Rates
df['exp_win1'] = (df['team1_adjoe']**11.5)/ ((df['team1_adjde']**11.5)+(df['team1_adjoe']**11.5))
df['exp_win2'] = (df['team2_adjoe']**11.5)/ ((df['team2_adjde']**11.5)+(df['team2_adjoe']**11.5))

In [13]:
# Function to calculate the distance between two teams  
def distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [14]:
df['dist1'] = df.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team1_lat'], row['team1_long']), axis=1)
df['dist2'] = df.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team2_lat'], row['team2_long']), axis=1)

In [15]:
df['diff_dist'] = df['dist1'] - df['dist2']

In [16]:
df['team1_win'] = (df['team1_score']>df['team2_score']).astype(int)

In [17]:
# Now, we can tell that labels have been successfully mixed 
df['team1_win'].value_counts()

0    650
1    596
Name: team1_win, dtype: int64

#### Drop IDs Variables 

We need to drop IDs, since it doesn't make any sense if we include in our dataset. 

ID在modeling中是没有用的，如果把它们放到模型中会被当成numerical variable影响预测 

In [18]:
df.drop(columns={"team1_id","team2_id","game_id","team2_teamname","team1_teamname","slot","team1_score","team2_score","team2_coach_id","team1_coach_id"},inplace=True) 

### Feature Selection (特征选择)

Since there are so many variables in the dataset provided. For a glimpse, we can tell a few issues. 

1. multicollinearity(多重共线性): 两个自变量直接高度相关, 会导致预测结果overfitting和coefficients不准确 (违反了linear regression的假设) 

2. Missing Value (缺失值): 有些自变量有很多缺失值，如果我们自己Impute的话会出现不准确影响预测 

3. Low variance (低方差): 自变量和因变量之间的方差低说明自变量和因变量直接的线性关系不明显，自变量没用

#### Missing Value Ration (缺失值)

In [19]:
# Keep the ratio of null values 
a = df.isnull().sum()/len(df)*100
# keep the feature names 
variables = df.columns
variable = [ ]

# Filter features if the ratio is above the threshold
for i in range(len(variables)):
    if a[i]<=20:   #setting the threshold as 10%
        variable.append(variables[i])

# Check how many variables were filtered 
print(len(variables) - len(variable))

8


#### Filter (过滤法) 
   ##### Low Variance Filter  
   
   We usually think that features with low variance carry very little information. We calculate the variance of all features, and we delete features under the threshold we set
   
Variance Inflation Factor (VIF) 的中文翻译是方差膨胀因子。它是一种评估多重共线性的统计学指标，用于衡量自变量对因变量的影响程度。VIF 值越大，说明自变量之间的共线性越强，因此通常需要减少自变量数量以提高模型的稳定性。

In [20]:
numeric = df.select_dtypes(include=np.number)

In [21]:
var = numeric.var()

numeric = numeric.columns
numeric_var = []

for i in range(0,len(var)):
    if var[i]>=10:   # 将阈值设置为10％
        numeric_var.append(numeric[i+1])

In [22]:
numeric_var.remove("team1_win") 

##### Variance Inflation Factor (因子膨胀分析） 

 Remove multicollinearities
 
To remove multicollinearities, we can do two things. We can create new features or remove them from our data.

Removing features is not recommended at first. The reason is that there’s a possibility of information loss because we remove that feature. Therefore, we will generate new features first.

From those features, we can generate the new one. The new feature will contain the difference value between those pairs. After we create those features, we can safely remove them from our data.

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()

x = df[numeric_var].dropna()
vif['feature'] = x.columns
vif['VIF'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

numeric_var2 = vif.loc[vif['VIF'] < 10]

# Display the VIF Result, we can tell that basically all features suffer from multicollinearity
print(numeric_var2)  

      feature       VIF
64  diff_dist  4.862231


C:\2345Downloads\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


#### L1 Regularization (Lasso正则化) 

We can tell that there are some pairs of features. 

As we know, L1 Regularization would be a great technique to select features to avoid multicollinearity.

Therefore, for the next part, we will try Regularization to find the most appropriate features.


L1正则化，也称为Lasso正则化，是机器学习和统计中使用的一种技术，它在损失函数中加入一个惩罚项，以限制模型的复杂度。 L1正则化通过限制系数的绝对值，来达到对结果的约束。它与L2正则化（Ridge正则化）的不同在于，L1正则化会产生稀疏解，即有些系数会变为0。

Ex: LogisticRegressionCV( penalty='l1')  

有时间看一下这俩视频: 

https://www.youtube.com/watch?v=Q81RR3yKn30 

https://www.youtube.com/watch?v=NGf0voTMlcs

##### Construct Train and test set 

In [24]:
y = df["team1_win"].astype("int")

In [25]:
print(numeric_var)

['team1_seed', 'strongseed', 'season', 'host_lat', 'host_long', 'team2_lat', 'team2_long', 'team1_lat', 'team1_long', 'team2_pt_school_ncaa', 'team2_pt_overall_ncaa', 'team2_pt_school_s16', 'team2_pt_overall_s16', 'team2_pt_school_ff', 'team2_pt_career_school_losses', 'team2_pt_career_overall_wins', 'team2_pt_career_overall_losses', 'team2_pt_team_season_wins', 'team2_pt_team_season_losses', 'team2_pt_coach_season_wins', 'team2_pt_coach_season_losses', 'team1_pt_school_ncaa', 'team1_pt_overall_ncaa', 'team1_pt_school_s16', 'team1_pt_overall_s16', 'team1_pt_school_ff', 'team1_pt_career_school_losses', 'team1_pt_career_overall_wins', 'team1_pt_career_overall_losses', 'team1_pt_team_season_wins', 'team1_pt_team_season_losses', 'team1_pt_coach_season_wins', 'team1_pt_coach_season_losses', 'team2_ap_preseason', 'team2_coaches_before_final', 'team2_coaches_preseason', 'team1_ap_final', 'team1_ap_preseason', 'team1_coaches_before_final', 'team1_coaches_preseason', 'team2_fg2pct', 'team2_block

In [26]:
categorical_var = []
remove = []

# The dependent variable team1_win have been removed from this loop 
for column in numeric:
    if column not in numeric_var:
            remove.append(column)

df.drop(columns=remove, inplace=True)

for column in df.columns:
    if column not in numeric_var:s
        categorical_var.append(column)

In [27]:
X = df.copy()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 

##### Buliding Pipeline for Logistic Regression CV


这个视频可以直观理解 Logistic Regression CV
https://www.youtube.com/watch?v=fSytzGwwBVw 

In [29]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler',StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numerical_transformer, numeric_var),
        ('cat', categorical_transformer, categorical_var)
    ])

In [30]:
# K - fold Cross Validation
kfolds = 5

# The range of penalty levels
min_alpha = 0.1
max_alpha = 100
n_candidates = 1000
C_list = list(1/np.linspace(min_alpha, max_alpha, num=n_candidates))

# Model Pipeline 
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', LogisticRegressionCV(Cs=C_list, cv=kfolds, penalty='l1',scoring='neg_log_loss',solver='liblinear', max_iter=2000, random_state=1, n_jobs=-1))
                             ])

In [31]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['team1_seed', 'strongseed',
                                                   'season', 'host_lat',
                                                   'host_long', 'team2_lat',
                                                   'team2_long', 'team1_lat',
                                                   'team1_long',
                                                   'team2_pt_school_ncaa',
                                                   'team2_pt_overall_ncaa',
                                        

In [32]:
preds = pipeline.predict_proba(X_test)[:,1]

print(metrics.log_loss(y_test, preds))

0.5881241183637567


In [34]:
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

##### Return a dataframe contains all feature names and coefficients 

For features returned coeficients of 0, we abandon these features for future model.

In [42]:
df_feature = pd.DataFrame(pipeline.named_steps['model'].coef_.flatten(), index=get_feature_names(preprocessor))

C:\Users\19493\AppData\Local\Temp/ipykernel_38564/1809191144.py:30: UserWarning: Transformer imputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
C:\Users\19493\AppData\Local\Temp/ipykernel_38564/1809191144.py:30: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


In [49]:
df_feature[0].value_counts()

 0.000000    132
-0.198781      1
-0.041571      1
-0.007171      1
 0.032801      1
 0.089930      1
-0.032672      1
-0.084550      1
-0.236445      1
 0.003284      1
 0.140342      1
-0.203006      1
-0.620468      1
-0.003772      1
Name: 0, dtype: int64

In [55]:
selected_features = df_feature[df_feature[0] != 0 ]

In [56]:
print(selected_features)

                                          0
num__team1_seed                   -0.198781
num__team2_pt_school_ncaa         -0.041571
num__team2_pt_team_season_wins    -0.007171
num__team2_pt_coach_season_losses  0.032801
num__team1_pt_overall_s16          0.089930
num__team2_ap_preseason           -0.032672
num__team2_blockpct               -0.084550
num__team2_adjoe                  -0.236445
num__team2_adjde                   0.003284
num__team1_oe                      0.140342
num__team1_adjde                  -0.203006
num__sead_diff                    -0.620468
num__diff_dist                    -0.003772


### Conclusion 

As we can see from select_feature dataframe, there are 13 features returned not 0 from Logistic Regression with l1 penalty. The logisticRegressionCV method gives us a comprehensive picture of what types of feature are most informative. 

The followings are features we selected with description.

(We did some minor changes, for example team1_oe and team1_adjoe are actually the same thing. The model returns the coef for team1_oe and abandon team1_adjoe ; however, for better understanding purpose, we choose team1_adjoe instead) 

Team Variables: 

1. sead_diff: The difference of seed between two schools. 

2. adjde: an estimate of the defense efficiency (points scored per 100 possessions) a team would have against the average D-I defense. 

3. adjoe: An estimate of the offensive efficiency (points scored per 100 possessions) a team would have against the average D-I defense.

4. blockpct - Blocked shots divided by opponents 2 point field goal attempts. 

5. ap_preseason: The preseason AP Poll ranking of each team (top 25 only) 

6. ap_final: The final AP Poll ranking of each team (top 25 only)

7. team_season_wins: Team’s number of wins in this season 

8. diff_dist: The distance between two schools 

Coach Variables: 

8. pt_overall_s16: Number of NCAA Sweet Sixteen appearances in entire career 

9. pt_coach_season_wins: Coach’s number of wins in this season

10. pt_school_ncaa: Number of NCAA Tournament appearances at current school

有时间看一下这几个gradient boost 

GradientBoost: https://www.youtube.com/watch?v=3CC4N4z3GJc&t=826s 

Xgboost: https://www.youtube.com/watch?v=8b1JEDvenQU 

为了accuracy 我们要使用组合模型： 

模型组合的方法包括：

Bagging（Bootstrapped Aggregating）：基于随机抽样的多个模型的简单平均，以减小单一模型的方差。

https://www.youtube.com/watch?v=2Mg8QD0F1dQ

Boosting：通过加强弱模型的权重以提高性能，最终通过加权平均以组合多个弱模型。

Stacking：通过使用第二层模型对多个第一层模型的预测进行组合。

https://www.youtube.com/watch?v=DCrcoh7cMHU

Adaboost：通过加强正确预测样本的权重，以提高弱模型的性能。

https://www.youtube.com/watch?v=LsK-xG1cLYA

Random Forest：通过使用随机抽样的决策树构建多个模型，并以简单平均或投票结果组合多个模型。

https://www.youtube.com/watch?v=dD7gvbfBiyA